## Lab 1: 간단한 고객 지원 에이전트 프로토타입 만들기

### 개요

[Amazon Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/)는 어떤 프레임워크나 모델을 사용하든 AI 에이전트를 안전하게 대규모로 배포하고 운영할 수 있도록 도와줍니다. 이를 통해 프로토타입에서 프로덕션으로 더 빠르게 전환할 수 있는 기능을 제공합니다. 

이 5단계 랩 튜토리얼에서는 **고객 지원 에이전트**를 사용하여 프로토타입에서 프로덕션까지의 전체 과정을 보여드립니다. 이 예제에서는 에이전트 구축을 위한 사용하기 쉬운 코드 우선 프레임워크인 [Strands Agents](https://strandsagents.com/latest/)와 Amazon Bedrock의 Anthropic Claude Haiku 4.5 모델을 사용합니다. 여러분의 애플리케이션에서는 원하는 프레임워크와 모델을 사용할 수 있습니다. 여기서 다루는 개념은 다른 프레임워크와 모델에도 적용될 수 있다는 점을 기억해 주세요.

**워크샵 여정:**
- **Lab 1 (현재)**: 에이전트 프로토타입 생성 - 기능적인 고객 지원 에이전트 구축
- **Lab 2**: 메모리 기능 강화 - 대화 맥락 및 개인화 추가
- **Lab 3**: 게이트웨이 및 아이덴티티로 확장 - 에이전트 간 도구를 안전하게 공유
- **Lab 4**: 프로덕션 배포 - 관측 기능을 갖춘 AgentCore 런타임 사용
- **Lab 5**: 에이전트 성능 평가 - 온라인 평가로 품질 모니터링
- **Lab 6**: 사용자 인터페이스 구축 - 고객 대면 애플리케이션 생성

이 첫 번째 랩에서는 워크샵 전체에 걸쳐 지속적인 메모리, 공유 도구, 전체 관측성을 갖춘 다수의 고객을 지원하는 프로덕션 준비 시스템으로 발전할 고객 지원 에이전트 프로토타입을 구축합니다. 에이전트는 다음과 같은 로컬 도구를 사용할 수 있습니다:
- **get_return_policy()** - 특정 제품에 대한 반품 정책 확인
- **get_product_info()** - 제품 정보 확인
- **web_search()** - 문제 해결 도움말을 위한 웹 검색
- **get_technical_support()** - Bedrock 기술 자료 검색


### Lab 1 아키텍처
<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*로컬에서 실행되는 간단한 프로토타입입니다. 후속 랩에서는 이를 공유 도구, 지속적인 메모리, 프로덕션 등급의 관측성을 갖춘 AgentCore 서비스로 마이그레이션할 것입니다.*

### 사전 필수 조건

* 적절한 권한이 있는 **AWS 계정**
* 로컬에 설치된 **Python 3.10+**
* 자격 증명으로 구성된 **AWS CLI**
* [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)에서 활성화된 **Anthropic Claude 3.7**
* 다음 셀에서 설치될 **Strands Agents** 및 기타 라이브러리

#### AWS 워크샵 계정을 사용하지 않으시나요? 

이 과정을 자습형 랩으로 진행하는 경우 CloudFormation 스택을 생성하고 배포하기 위해 추가로 2단계를 수행해야 합니다:

**단계 0.1: [자습형 랩 전용]** 

아래 셀의 `!aws sts get-caller-identity` 명령 주석을 해제하여 Sagemaker 역할을 확인하세요. AWS 콘솔의 IAM으로 이동하여 Sagemaker 역할을 검색합니다. 이제 [워크샵 자습형 사전 요구 사항](https://catalog.us-east-1.prod.workshops.aws/workshops/850fcd5c-fd1f-48d7-932c-ad9babede979/en-US/00-prerequisites/02-self-paced) 랩에 설명된 대로 [IAM 정책, AWS 관리형 정책 및 신뢰 관계](https://catalog.us-east-1.prod.workshops.aws/workshops/850fcd5c-fd1f-48d7-932c-ad9babede979/en-US/00-prerequisites/02-self-paced#iam-policy-for-bedrock-agentcore-workshop)를 추가하세요.

In [ ]:
# Note: Uncomment and run only for self-paced labs
# !aws sts get-caller-identity 

**단계 0.2: [자습형 랩 전용]**

이제 `prereq.sh` 스크립트에 필요한 모든 권한을 갖추었으면 다음 명령을 실행하여 CloudFormation 템플릿을 배포하세요.

In [ ]:
# Note: Uncomment and run only for self-paced labs
# !bash scripts/prereq.sh

### 단계 1: 의존성 설치 및 라이브러리 가져오기
시작하기 전에 이 랩에 필요한 의존성을 설치해 봅시다. 몇 가지 의존성 오류가 표시될 수 있지만, 이 워크샵 범위 내에서는 무시해도 안전합니다.

In [ ]:
# Install required packages 
%pip install -U -r requirements.txt -q

이제 필요한 라이브러리를 가져오고 boto3 세션을 초기화할 수 있습니다.

In [ ]:
# Import libraries
import boto3
from boto3.session import Session

from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

from strands.tools import tool

In [ ]:
# Get boto session
boto_session = Session()
region = boto_session.region_name

### 단계 2: 사용자 정의 도구 구현

다음으로, 고객 지원 에이전트에게 제공될 3가지 도구를 구현해 보겠습니다.

Strands Agent에서 도구를 정의하는 것은 매우 간단합니다. 함수에 `@tool` 데코레이터를 추가하고 함수의 독스트링에 도구 설명을 제공하면 됩니다. Strands Agents는 함수 문서, 타이핑, 인수를 사용하여 에이전트에게 이 도구에 대한 컨텍스트를 제공합니다. 


#### 도구 1: 반품 정책 확인

**목적:** 이 도구는 고객이 다양한 제품 카테고리의 반품 정책을 이해하도록 돕습니다. 반품 기간, 조건, 절차 및 환불 일정에 대한 자세한 정보를 제공하여 고객이 제품을 반품할 때 무엇을 예상해야 하는지 정확히 알 수 있게 합니다.

In [ ]:
@tool
def get_return_policy(product_category: str) -> str:
    """
    Get return policy information for a specific product category.

    Args:
        product_category: Electronics category (e.g., 'smartphones', 'laptops', 'accessories')

    Returns:
        Formatted return policy details including timeframes and conditions
    """
    # Mock return policy database - in real implementation, this would query policy database
    return_policies = {
        "smartphones": {
            "window": "30 days",
            "condition": "Original packaging, no physical damage, factory reset required",
            "process": "Online RMA portal or technical support",
            "refund_time": "5-7 business days after inspection",
            "shipping": "Free return shipping, prepaid label provided",
            "warranty": "1-year manufacturer warranty included",
        },
        "laptops": {
            "window": "30 days",
            "condition": "Original packaging, all accessories, no software modifications",
            "process": "Technical support verification required before return",
            "refund_time": "7-10 business days after inspection",
            "shipping": "Free return shipping with original packaging",
            "warranty": "1-year manufacturer warranty, extended options available",
        },
        "accessories": {
            "window": "30 days",
            "condition": "Unopened packaging preferred, all components included",
            "process": "Online return portal",
            "refund_time": "3-5 business days after receipt",
            "shipping": "Customer pays return shipping under $50",
            "warranty": "90-day manufacturer warranty",
        },
    }

    # Default policy for unlisted categories
    default_policy = {
        "window": "30 days",
        "condition": "Original condition with all included components",
        "process": "Contact technical support",
        "refund_time": "5-7 business days after inspection",
        "shipping": "Return shipping policies vary",
        "warranty": "Standard manufacturer warranty applies",
    }

    policy = return_policies.get(product_category.lower(), default_policy)
    return (
        f"Return Policy - {product_category.title()}:\n\n"
        f"• Return window: {policy['window']} from delivery\n"
        f"• Condition: {policy['condition']}\n"
        f"• Process: {policy['process']}\n"
        f"• Refund timeline: {policy['refund_time']}\n"
        f"• Shipping: {policy['shipping']}\n"
        f"• Warranty: {policy['warranty']}"
    )


print("✅ Return policy tool ready")

#### 도구 2: 제품 정보 확인

**목적:** 이 도구는 보증, 사용 가능한 모델, 주요 기능, 배송 정책 및 반품 정보를 포함한 포괄적인 제품 세부 정보를 고객에게 제공합니다. 고객이 정보에 입각한 구매 결정을 내리고 구매하는 제품을 이해하는 데 도움을 줍니다.

In [ ]:
@tool
def get_product_info(product_type: str) -> str:
    """
    Get detailed technical specifications and information for electronics products.

    Args:
        product_type: Electronics product type (e.g., 'laptops', 'smartphones', 'headphones', 'monitors')
    Returns:
        Formatted product information including warranty, features, and policies
    """
    # Mock product catalog - in real implementation, this would query a product database
    products = {
        "laptops": {
            "warranty": "1-year manufacturer warranty + optional extended coverage",
            "specs": "Intel/AMD processors, 8-32GB RAM, SSD storage, various display sizes",
            "features": "Backlit keyboards, USB-C/Thunderbolt, Wi-Fi 6, Bluetooth 5.0",
            "compatibility": "Windows 11, macOS, Linux support varies by model",
            "support": "Technical support and driver updates included",
        },
        "smartphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "5G/4G connectivity, 128GB-1TB storage, multiple camera systems",
            "features": "Wireless charging, water resistance, biometric security",
            "compatibility": "iOS/Android, carrier unlocked options available",
            "support": "Software updates and technical support included",
        },
        "headphones": {
            "warranty": "1-year manufacturer warranty",
            "specs": "Wired/wireless options, noise cancellation, 20Hz-20kHz frequency",
            "features": "Active noise cancellation, touch controls, voice assistant",
            "compatibility": "Bluetooth 5.0+, 3.5mm jack, USB-C charging",
            "support": "Firmware updates via companion app",
        },
        "monitors": {
            "warranty": "3-year manufacturer warranty",
            "specs": "4K/1440p/1080p resolutions, IPS/OLED panels, various sizes",
            "features": "HDR support, high refresh rates, adjustable stands",
            "compatibility": "HDMI, DisplayPort, USB-C inputs",
            "support": "Color calibration and technical support",
        },
    }
    product = products.get(product_type.lower())
    if not product:
        return f"Technical specifications for {product_type} not available. Please contact our technical support team for detailed product information and compatibility requirements."

    return (
        f"Technical Information - {product_type.title()}:\n\n"
        f"• Warranty: {product['warranty']}\n"
        f"• Specifications: {product['specs']}\n"
        f"• Key Features: {product['features']}\n"
        f"• Compatibility: {product['compatibility']}\n"
        f"• Support: {product['support']}"
    )


print("✅ get_product_info tool ready")

#### 도구 3: 웹 검색

**목적:** 이 도구는 고객이 문제 해결 지원을 받거나 제품 추천 등에 대한 제안을 받을 수 있게 해줍니다.

In [ ]:
@tool
def web_search(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """Search the web for updated information.

    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.

    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "Rate limit reached. Please try again later."
    except DDGSException as e:
        return f"Search error: {e}"
    except Exception as e:
        return f"Search error: {str(e)}"


print("✅ Web search tool ready")

#### 고객 지원 에이전트 - 기술 자료(Knowledge Base) 통합 단계

##### S3에서 제품 기술 지원 파일 다운로드

In [ ]:
import os


def download_files():
    # Get account and region
    account_id = boto3.client("sts").get_caller_identity()["Account"]
    region = boto3.Session().region_name
    bucket_name = f"{account_id}-{region}-kb-data-bucket"

    # Create local folder
    os.makedirs("knowledge_base_data", exist_ok=True)

    # Download all files
    s3 = boto3.client("s3")
    objects = s3.list_objects_v2(Bucket=bucket_name)

    for obj in objects["Contents"]:
        file_name = obj["Key"]
        s3.download_file(bucket_name, file_name, f"knowledge_base_data/{file_name}")
        print(f"Downloaded: {file_name}")

    print("All files saved to: knowledge_base_data/")


# Run it
download_files()

#### Knowledge Base 동기화 작업

##### 에이전트와 통합할 수 있는 S3의 제품 기술 지원 파일로 기술 자료 동기화

In [ ]:
import time

# Get parameters
ssm = boto3.client("ssm")
bedrock = boto3.client("bedrock-agent")
s3 = boto3.client("s3")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/knowledge-base-id")[
    "Parameter"
]["Value"]
ds_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/data-source-id")[
    "Parameter"
]["Value"]

# Get file names from S3 bucket
bucket_name = f"{account_id}-{region}-kb-data-bucket"
s3_objects = s3.list_objects_v2(Bucket=bucket_name)
file_names = [obj["Key"] for obj in s3_objects.get("Contents", [])]

# Start sync job
response = bedrock.start_ingestion_job(
    knowledgeBaseId=kb_id, dataSourceId=ds_id, description="Quick sync"
)

job_id = response["ingestionJob"]["ingestionJobId"]
print("Bedrock knowledge base sync job started, ingesting the data files from s3")

# Monitor until complete
while True:
    job = bedrock.get_ingestion_job(
        knowledgeBaseId=kb_id, dataSourceId=ds_id, ingestionJobId=job_id
    )["ingestionJob"]

    status = job["status"]

    if status in ["COMPLETE", "FAILED"]:
        break

    time.sleep(10)

# Print final result
if status == "COMPLETE":
    file_count = job.get("statistics", {}).get("numberOfDocumentsScanned", 0)
    files_list = ", ".join(file_names)
    print(
        f"Bedrock knowledge base sync job completed Successfully, ingested {file_count} files"
    )
    print(f"Files ingested: {files_list}")
else:
    print(f"Bedrock knowledge base sync job failed with status: {status}")

#### 도구 4: 기술 지원 받기

**목적:** 이 도구는 전자 제품 문서 기술 자료에 액세스하여 고객에게 포괄적인 기술 지원 및 문제 해결 지원을 제공합니다. 자세한 설정 가이드, 유지 관리 지침, 문제 해결 단계, 연결 솔루션 및 보증 서비스 정보가 포함되어 있습니다. 이 도구는 고객이 기술적인 문제를 해결하고, 기기를 올바르게 구성하며, 최적의 제품 성능을 위한 유지 관리 요구 사항을 이해하는 데 도움을 줍니다.

In [ ]:
from strands.models import BedrockModel
from strands import Agent
from strands_tools import retrieve


@tool
def get_technical_support(issue_description: str) -> str:
    try:
        # Get KB ID from parameter store
        ssm = boto3.client("ssm")
        account_id = boto3.client("sts").get_caller_identity()["Account"]
        region = boto3.Session().region_name

        kb_id = ssm.get_parameter(Name=f"/{account_id}-{region}/kb/knowledge-base-id")[
            "Parameter"
        ]["Value"]
        print(f"Successfully retrieved KB ID: {kb_id}")

        # Use strands retrieve tool
        tool_use = {
            "toolUseId": "tech_support_query",
            "input": {
                "text": issue_description,
                "knowledgeBaseId": kb_id,
                "region": region,
                "numberOfResults": 3,
                "score": 0.4,
            },
        }

        result = retrieve.retrieve(tool_use)

        if result["status"] == "success":
            return result["content"][0]["text"]
        else:
            return f"Unable to access technical support documentation. Error: {result['content'][0]['text']}"

    except Exception as e:
        print(f"Detailed error in get_technical_support: {str(e)}")
        return f"Unable to access technical support documentation. Error: {str(e)}"


print("✅ Technical support tool ready")

### 단계 4: 고객 지원 에이전트 생성 및 구성

다음으로, 모델, 이전 단계에서 구현한 도구 목록, 그리고 시스템 프롬프트를 제공하여 고객 지원 에이전트를 생성하겠습니다.

In [ ]:
SYSTEM_PROMPT = """You are a helpful and professional customer support assistant for an electronics e-commerce company.
Your role is to:
- Provide accurate information using the tools available to you
- Support the customer with technical information and product specifications, and maintenance questions
- Be friendly, patient, and understanding with customers
- Always offer additional help after answering questions
- If you can't help with something, direct customers to the appropriate contact

You have access to the following tools:
1. get_return_policy() - For warranty and return policy questions
2. get_product_info() - To get information about a specific product
3. web_search() - To access current technical documentation, or for updated information. 
4. get_technical_support() - For troubleshooting issues, setup guides, maintenance tips, and detailed technical assistance
For any technical problems, setup questions, or maintenance concerns, always use the get_technical_support() tool as it contains our comprehensive technical documentation and step-by-step guides.

Always use the appropriate tool to get accurate, up-to-date information rather than making assumptions about electronic products or specifications."""

# Initialize the Bedrock model (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.3,
    region_name=region,
)

# Create the customer support agent with all tools
agent = Agent(
    model=model,
    tools=[
        get_product_info,  # Tool 1: Simple product information lookup
        get_return_policy,  # Tool 2: Simple return policy lookup
        web_search,  # Tool 3: Access the web for updated information
        get_technical_support,  # Tool 4: Technical support & troubleshooting
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("Customer Support Agent created successfully!")

### 단계 5: 고객 지원 에이전트 테스트

에이전트가 모든 도구를 올바르게 사용하는지 확인하기 위해 샘플 쿼리로 테스트해 봅시다.

#### 반품 확인 테스트

In [ ]:
response = agent("What's the return policy for my thinkpad X1 Carbon?")

In [ ]:
response = agent("My laptop won't turn on, what should I check?")

#### 문제 해결 테스트

In [ ]:
response = agent(
    "I bought an iphone 14 last month. I don't like it because it heats up. How do I solve it?"
)

## 🎉 Lab 1 완료!

기능적인 고객 지원 에이전트 프로토타입을 성공적으로 만들었습니다! 달성한 내용은 다음과 같습니다:

- 3가지 맞춤형 도구(반품 정책, 제품 정보, 웹 검색)를 갖춘 에이전트 구축  
- 다중 도구 상호 작용 및 웹 검색 기능 테스트  
- 프로덕션 여정을 위한 기반 마련  

### 현재의 한계 (우리가 해결할 것입니다!)
- **단일 사용자 대화 메모리** - 로컬 대화 세션, 여러 고객은 여러 세션이 필요합니다.
- **세션으로 제한된 대화 기록** - 대화에서 장기 메모리나 교차 세션 정보를 사용할 수 없습니다.
- **도구 재사용성** - 도구를 다른 에이전트 간에 재사용할 수 없습니다  
- **로컬에서만 실행** - 확장 불가능
- **아이덴티티** - 사용자 및/또는 에이전트 아이덴티티나 액세스 제어가 없습니다
- **관측성** - 에이전트 동작에 대한 관측성이 제한적입니다
- **기존 API** - 고객 데이터를 위한 기존 엔터프라이즈 API에 액세스할 수 없습니다

##### 다음 [Lab 2: 메모리를 추가하여 에이전트 개인화하기 →](lab-02-agentcore-memory.ipynb)
